In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/pbp_related/"
data_DIR = "../data/pbpdata/"
source = "pbpstats"
exp_DIR = "../data/misc/"
box_DIR = "../data/box/"

from pbpstats.resources.enhanced_pbp import Timeout

In [ ]:
league = "NBA"
season = "2023"
year = season + "-" + str(int(season)+1)[-2:]
with zstd.open(data_DIR + league+"_PBPdata_" + season + ".pkl.zst","rb") as f:
    games_list = dill.load(f)
player_dict = get_players_pbp(league=league)  # Creating Player Dictionary
team_dict, team_list = get_teams(league=league)  # Creating Team Dictionary

In [ ]:
dft = pd.read_parquet(box_DIR + "NBA_Box_T_Base_" + season + ".parquet")

In [ ]:
df_bucks = dft.query("TEAM_ABBREVIATION == 'MIL'").reset_index()

In [ ]:
bf_glenn = df_bucks["GAME_ID"].iloc[:46].to_list()
af_glenn = df_bucks["GAME_ID"].iloc[46:].to_list()

In [ ]:
df_ft = pd.read_parquet(box_DIR + "NBA_Box_P_Cum_Base_" + season + ".parquet", columns = ["PLAYER_ID","FT_PCT"])
df_ft = df_ft.set_index("PLAYER_ID")
ft_dict = df_ft.to_dict()
ft_dict = ft_dict["FT_PCT"]

In [ ]:
games_list_bf = [game for game in games_list  if game.game_id in bf_glenn]
games_list_af = [game for game in games_list  if game.game_id in af_glenn]

In [ ]:
len(games_list_bf)

In [ ]:
possessions = [game.possessions.items for game in games_list_af]
possession_events = list(chain(*[possession.events for possession in list(chain(*possessions))]))

In [ ]:
poss = []
idx = []
for i,possession_event in enumerate(possession_events):
    if (
        isinstance(possession_event, Timeout)
    ):
        idx.append(i)
idx.append(len(possession_events))

In [ ]:
idxi = []
idxj = []
for j in range(len(idx)-1):
    for ii in range(idx[j]+1,idx[j+1]):
        poss = possession_events[ii]
        if ((isinstance(poss, FieldGoal)) or (isinstance(poss, Turnover))) or (isinstance(poss, Foul) and poss.is_shooting_foul):
            idxi.append(ii)
            idxj.append(idx[j])
            break

In [ ]:
i = 20
poss1 = possession_events[idxj[i]] 
poss2 = possession_events[idxi[i]]
print(poss1)
print(poss2)
print(poss2.team_id)

In [ ]:
pts = []
team = []
possa = []
possc = []
for i in idxi:
    poss = possession_events[i]
    
    possa.append(poss.description)
    if isinstance(poss, FieldGoal):
        team.append(poss.team_id)
        if poss.is_made:
            pts.append(poss.opt1)
        else:
            pts.append(0)
    elif isinstance(poss, Turnover):
        team.append(poss.team_id)
        pts.append(0)
    elif isinstance(poss, Foul):
        team.append(poss.offense_team_id)
        fta = poss.number_of_fta_for_foul
        ftpct = ft_dict[poss.player3_id]
        pts.append(fta*ftpct)
        possc.append(poss)
possb = []
for i in idxj:
    poss = possession_events[i]
    possb.append(poss.description)

In [ ]:
df = pd.DataFrame({"teamID":team,"pts":pts,"timeout":possb,"desc":possa})

In [ ]:
df["team"] = df["teamID"].map(team_dict)

In [ ]:
df = df[["team","pts"]]

In [ ]:
dfg = df.groupby("team")[["pts"]].agg({"pts":["count","sum"]})

In [ ]:
dfg = dfg.reset_index()
dfg.columns = ["team","poss","pts"]

In [ ]:
dfg["ORtg"] = round(dfg["pts"]/dfg["poss"],2)

In [ ]:
dfg = dfg.sort_values(by=["ORtg"],ascending=False).reset_index(drop=True)
dfg.index +=1

In [ ]:
dfg